In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import pandas as pd
import tensorflow as tf

from edward.models import Bernoulli, MultivariateNormalTriL, Normal
from edward.util import rbf

import sklearn
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler,QuantileTransformer
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

C:\Users\Rhea\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ed.set_seed(42)

In [3]:
data=pd.read_csv("C:\\Users\\Rhea\\Documents\\Bayesian Modeling\\Datasets\\BreastCancerWisconsin\\breast-cancer-wisconsin.data",
names=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean',
       'concave_points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se',
       'compactness_se','concavity_se','concave_points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst',
       'perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave_points_worst',
       'symmetry_worst','fractal_dimension_worst'])

In [4]:
data.shape

(569, 32)

In [5]:
data['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

#### Convert relevant columns to categorical

In [6]:
def change_to_category(df):
    i=0;
    while i<df.columns.size:
        if ((df.dtypes[df.columns[i]] == 'object' or df.dtypes[df.columns[i]] == 'bool')):
            col = df.dtypes.index[i]
            df[col] = df[col].astype('category')
        i = i+1 

#### Generate X and y variables

In [7]:
def generateXY(df,target_col,var_list):
    X=df[varToUse]
    y=df[target_col]
    return X,y

Since we aim to classify patients as having Malignant and Benign tumor, we decide **'diagnosis'** to be our target variable.
<br>
Also, we replace 'M' with 1 and 'B' with 0, in the column

In [8]:
# data['diagnosis'] = data['diagnosis'].replace('M', 1)
# data['diagnosis'] = data['diagnosis'].replace('B', 0)
data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})

In [9]:
# Convert columns with binary values to categorical
change_to_category(data)

In [10]:
data.diagnosis.dtype

dtype('int64')

In [11]:
data.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave_points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave_points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [12]:
#drop irrelevant columns
data.drop(['id'], axis = 1, inplace = True)

In [13]:
data2 = pd.get_dummies(data)
data2.shape

(569, 31)

In [14]:
# create training and testing vars
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)


In [15]:
#Indeppendent and Dependent features
target_col='diagnosis'
varToUse=data.columns.tolist()
varToUse.remove('diagnosis')


Train_X,Train_Y = generateXY(train,target_col,varToUse)
Test_X,Test_Y = generateXY(test,target_col,varToUse)

In [16]:
from sklearn import svm
    print ("Model: Support Vector Machines")
svm_clf = svm.SVC(kernel='linear', C = 1.0)

precision_svm = cross_val_score(svm_clf, Train_X, Train_Y, cv=10, scoring='precision')
print ("Train Precision = {}".format(precision_svm.mean()))
recall_svm = cross_val_score(svm_clf, Train_X, Train_Y, cv=10, scoring='recall')
print ("Train Recall = {}".format(recall_svm.mean()))
f1score_svm = cross_val_score(svm_clf, Train_X, Train_Y, cv=10, scoring='f1')
print ("Train F1 Score  = {}".format(f1score_svm.mean()))
accscore_svm = cross_val_score(svm_clf, Train_X, Train_Y, cv=10, scoring='accuracy', n_jobs=-1)
print ("Accuracy score  = {}".format(accscore_svm.mean()))

Model: Support Vector Machines
Train Precision = 0.9517273576097105
Train Recall = 0.9102941176470589
Train F1 Score  = 0.9285341350616495
Accuracy score  = 0.9490206411945541


In [17]:
svm_clf.fit(X=Train_X,y=Train_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
y_pred_svm=svm_clf.predict(Test_X)

In [19]:
svm_report=sklearn.metrics.classification_report(Test_Y,y_pred_svm)
print(svm_report)
print("confusion matrix")
print(confusion_matrix(Test_Y, y_pred_svm))
print("accuracy score")
print(accuracy_score(Test_Y, y_pred_svm, normalize=True, sample_weight=None))

             precision    recall  f1-score   support

          0       0.95      0.99      0.97        71
          1       0.97      0.91      0.94        43

avg / total       0.96      0.96      0.96       114

confusion matrix
[[70  1]
 [ 4 39]]
accuracy score
0.956140350877


In [20]:
data_x = data2.iloc[0:,1:].as_matrix().astype(np.float32)
data_y = data2.iloc[0:,0:1].as_matrix().astype(np.float32)

N = 300
train_x, test_x = data_x[:N], data_x[N:]
train_y, test_y = data_y[:N], data_y[N:]

in_size = train_x.shape[1]
out_size = train_y.shape[1]

N = train_x.shape[0]  # number of data points
D = train_x.shape[1]  # number of features


In [21]:
print("Number of data points in train: {}".format(N))
print("Number of features in train: {}".format(D))

Number of data points in train: 300
Number of features in train: 30


In [22]:
X = tf.placeholder(tf.float32, [N, D])
f = MultivariateNormalTriL(loc=tf.zeros(N), scale_tril=tf.cholesky(rbf(X)))
y = Bernoulli(logits=f)

In [23]:
qf = Normal(loc=tf.get_variable("qf/loc", [N]),
            scale=tf.nn.softplus(tf.get_variable("qf/scale", [N])))

In [24]:
train_y.shape

(300, 1)

In [25]:
inference = ed.KLqp({f: qf}, data={X: train_x})
inference.run(n_iter=5000)

5000/5000 [100%] ██████████████████████████████ Elapsed: 54s | Loss: 0.165
